In [5]:
import pandas as pd
import numpy as np

In [22]:
datosDemanda = pd.read_csv("Demand13.csv")
datosProduccion = pd.read_csv("Supply13.csv")

In [23]:
datosDemanda["total"] = datosDemanda.sum(axis=1)
datosProduccion["total"] = datosProduccion.sum(axis=1)

In [10]:
print(datosDemanda)
datosDemanda.head()

       Hours  Baghdad  Nynwa  Kirkuk  Slah El-Din  Anbar  Dyala  Babil  Kerb.  \
0          1     3294   1243     757          488    471    598    525    514   
1          2     2973   1151     753          477    436    570    500    469   
2          3     2758   1105     727          471    419    604    478    430   
3          4     2703   1220     717          470    463    564    497    465   
4          5     2954   1358     751          519    515    626    553    517   
...      ...      ...    ...     ...          ...    ...    ...    ...    ...   
26347     20     4894   1906    1206         1008    912   1112   1112    914   
26348     21     4550   1798    1158          928    862   1047   1027    869   
26349     22     4326   1683    1091          868    818   1001    972    807   
26350     23     4090   1567     988          819    782    935    892    780   
26351     24     3954   1465     942          780    750    879    838    728   

       Najaf  Dywan.  Wasit

,Hours,Baghdad,Nynwa,Kirkuk,Slah El-Din,Anbar,Dyala,Babil,Kerb.,Najaf,Dywan.,Wasit,Muthnna,Thi-Qar,Mesan,Basrah,total
0,1,3294,1243,757,488,471,598,525,514,474,307,404,208,573,369,1254,11480
1,2,2973,1151,753,477,436,570,500,469,437,284,374,193,536,342,1100,10597
2,3,2758,1105,727,471,419,604,478,430,420,273,359,185,519,328,1056,10135
3,4,2703,1220,717,470,463,564,497,465,451,301,397,204,522,363,1100,10441
4,5,2954,1358,751,519,515,626,553,517,502,335,442,228,574,404,1298,11581


In [24]:
df1 = datosDemanda[["Hours", "total"]]

In [25]:
df_grouped = df1.groupby(df1["Hours"]//24).sum()

In [26]:
print(df_grouped)

        Hours      total
Hours                   
0      303048  443929019
1       26352   18659914


In [19]:
days = pd.date_range(start='1/1/2019', periods=len(df_grouped), freq='D')

In [20]:
df_grouped['fecha'] = days

In [21]:
print(df_grouped)

        Hours      total      fecha
Hours                              
0      303048  887858038 2019-01-01
1       26352   37319828 2019-01-02
